## Training CNN or DNN on MNIST and Noisy MNIST dataset (n-MNIST)

- A DNN network
- A Simple CNN network
- A Normal CNN network

In [43]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

### The neuros setting of hidden layer

In [44]:
neuros = 500
# neuros = 200

### 1, Deep Neural Network (DNN)

In [45]:
class DNNNet(nn.Module):
    def __init__(self):
        super(DNNNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, neuros)
        self.fc2 = nn.Linear(neuros, 10)

    def forward(self, x):

        x = x.view(-1, 28 * 28 * 1)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

### 2, Simple Convolutional Neural Network(SCNN)

In [46]:
class SimpleCNNNet(nn.Module):
    def __init__(self):
        super(SimpleCNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.fc1 = nn.Linear(12 * 12 * 20, neuros)
        self.fc2 = nn.Linear(neuros, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 12 * 12 * 20)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

### 3, Convolutional Neural Network (CNN)

In [47]:
class CNNNet(nn.Module):

    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, neuros)
        self.fc2 = nn.Linear(neuros, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))

        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

### Training Models

In [48]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if False:
                break

In [49]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return round((100. * correct / len(test_loader.dataset)),3)

#### 1, Training the models on MNIST datasets

In [57]:
#!/usr/bin/env python
"""
Train a basic MNIST CNN.

Based on the PyTorch example code here:
https://github.com/pytorch/examples/blob/master/mnist/main.py
"""
from __future__ import print_function

import os
import sys
import argparse
import torch

import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [59]:
net_list = ['dnn', 'scnn', 'cnn']
accuracys = []

learning_rate = 0.01
step_size=1
gamma=0.7
momentum = 0.5
epoch_num = 10
seed = 1
train_batch_size = 64
test_batch_size = 1000

for net in net_list:
    print('=====================', net,'======================')
    # Training settings
    use_cuda = not False and torch.cuda.is_available()
    accuracy_list = []
    torch.manual_seed(seed)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    kwargs = {}
    if use_cuda:
        torch.cuda.manual_seed(seed)
        kwargs = {'num_workers': 1,'pin_memory': True}
    
    train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=True, download=True,
           transform=transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=train_batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=False,download=True,
           transform=transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=test_batch_size, shuffle=True, **kwargs)
    

    if net == 'dnn':
        model = DNNNet().to(device)
    elif net == 'scnn':
        model = SimpleCNNNet().to(device)
    elif net == 'cnn':
        model = CNNNet().to(device)

    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    for epoch in range(1, epoch_num + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        acc = test(model, device, test_loader)
        accuracy_list.append(acc)

    if True:
        if net == 'dnn':
            torch.save(model.state_dict(), "mnist_dnn_net_"+str(neuros)+".pt")
        elif net == 'scnn':
            torch.save(model.state_dict(), "mnist_cnn_net_simple_"+str(neuros)+".pt")
        elif net == 'cnn':
            torch.save(model.state_dict(), "mnist_cnn_net_"+str(neuros)+".pt")
    print(accuracy_list)
    accuracys.append(accuracy_list)

print(accuracys)

===================== dnn ======================
Failed to download (trying next):
HTTP Error 503: Service Unavailable



92.1%

KeyboardInterrupt: 

#### Show the testing loss value

In [ ]:
accuracys

In [ ]:
x = range(1, epoch_num+1)
plt.figure(figsize=(12,6))
y1 = accuracys[0]
y2 = accuracys[1]
y3 = accuracys[2]
plt.plot(x, y1, label='DNN', linestyle='dotted')
plt.plot(x, y2, label='Simple CNN', linestyle='dashdot')
plt.plot(x, y3, label='CNN')
ax = plt.subplot()
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))
plt.yticks(np.arange(95, 100, 0.5))
plt.xticks(np.arange(0, 16, 1))
plt.legend()
plt.xlabel('Epochs', fontsize='14')
plt.ylabel('Accuracy(%)', fontsize='14')
# plt.title('The accuracy of the different depth neural network')
plt.show()

#### 2,Training the models on Noisy MNIST dataset

In [15]:
from __future__ import print_function

import os
import errno
import scipy.io as sio
import numpy as np
import torch
from torchvision.datasets import MNIST
from torchvision.datasets.utils import download_url

In [16]:
class NMNIST(MNIST):
    """`n-MNIST <http://www.csc.lsu.edu/~saikat/n-mnist/>`_ Dataset.
    Args:
        root (string): Root directory of dataset where ``mnist-with-awgn.mat``,
            ``mnist-with-motion-blur.mat`` and
            ``mnist-with-reduced-contrast-and-awgn.mat`` exist.
        train (bool, optional): If True, loads training data, otherwise loads
            test data.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        dataset (string, optional): Select the particular n-MNIST dataset to load:
            "awgn", "motion-blur", "reduced-contrast-and-awgn" or "all".
    """
    urls = {
        'awgn': 'http://www.csc.lsu.edu/~saikat/n-mnist/data/mnist-with-awgn.gz',
        'motion-blur': 'http://www.csc.lsu.edu/~saikat/n-mnist/data/mnist-with-motion-blur.gz',
        'reduced-contrast-and-awgn': 'http://www.csc.lsu.edu/~saikat/n-mnist/data/mnist-with-reduced-contrast-and-awgn.gz',
    }

    def __init__(self, root, train=True, transform=None, target_transform=None, download=False, dataset='awgn'):
        self.dataset = dataset

        if self.dataset != 'all':
            self.urls = {self.dataset: self.urls[self.dataset]}

        self.gzip_files = []
        self.mat_files = []
        for _, url in self.urls.items():
            self.gzip_files.append(os.path.basename(url))
            self.mat_files.append(os.path.splitext(os.path.basename(url))[0] + '.mat')

        self.training_file = self.dataset + '-training.pt'
        self.test_file = self.dataset + '-test.pt'

        super(NMNIST, self).__init__(root, train=train, transform=transform, target_transform=target_transform, download=download)

    def _check_gzips_exists(self):
        for gzip_file in self.gzip_files:
            if not os.path.exists(os.path.join(self.root, self.raw_folder, gzip_file)):
                return False
        return True

    def _check_mats_exists(self):
        for mat_file in self.mat_files:
            if not os.path.exists(os.path.join(self.root, self.raw_folder, mat_file)):
                return False
        return True

    def download(self):
        """Download the n-MNIST data if it doesn't exist in processed_folder already."""
        from six.moves import urllib
        import tarfile

        if self._check_exists():
            return

        # download files
        try:
            os.makedirs(os.path.join(self.root, self.raw_folder))
            os.makedirs(os.path.join(self.root, self.processed_folder))
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise

        if not self._check_mats_exists():
            for _, url in self.urls.items():
                filename = url.rpartition('/')[2]
                file_path = os.path.join(self.root, self.raw_folder, filename)
                if not self._check_gzips_exists():
                    download_url(url, root=os.path.join(self.root, self.raw_folder),
                                 filename=filename, md5=None)
                with open(file_path.replace('.gz', '.mat'), 'wb') as out_f:
                    tar = tarfile.open(file_path, 'r:gz')
                    zip_f = tar.extractfile(os.path.basename(file_path.replace('.gz', '.mat')))
                    out_f.write(zip_f.read())
                    os.unlink(file_path)

        # process and save as torch files
        print('Processing...')

        def read_images(mat_data, split):
            length = mat_data[split].shape[0]
            num_rows = np.uint8(np.sqrt(mat_data[split].shape[1]))
            num_cols = num_rows
            return torch.from_numpy(mat_data[split]).view(length, num_rows, num_cols)

        def read_labels(mat_data, split):
            length = mat_data[split].shape[0]
            labels = np.asarray([np.where(r == 1)[0][0] for r in mat_data[split]])
            return torch.from_numpy(labels).view(length).long()

        data = sio.loadmat(os.path.join(self.root, self.raw_folder, self.mat_files[0]))
        if len(self.mat_files) > 1:
            for mat_file in self.mat_files[1:]:
                mat_data = sio.loadmat(os.path.join(self.root, self.raw_folder, mat_file))
                data['train_x'] = np.concatenate((data['train_x'], mat_data['train_x']), axis=0)
                data['train_y'] = np.concatenate((data['train_y'], mat_data['train_y']), axis=0)
                data['test_x'] = np.concatenate((data['test_x'], mat_data['test_x']), axis=0)
                data['test_y'] = np.concatenate((data['test_y'], mat_data['test_y']), axis=0)

        training_set = (
            read_images(data, 'train_x'),
            read_labels(data, 'train_y')
        )
        test_set = (
            read_images(data, 'test_x'),
            read_labels(data, 'test_y')
        )
        with open(os.path.join(self.root, self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.root, self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')

In [34]:
from os.path import dirname, realpath
root_path = dirname(os.getcwd())
sys.path.append(root_path)
print(root_path)

/Users/shengdaolin_sh/dir_developer/workspaces/pycharm/acs-project-msc_project_ndp/ndp/mnist_cnn


In [35]:
net_list = ['dnn', 'scnn', 'cnn']
accuracys = []

learning_rate = 0.01
step_size=1
gamma=0.7
momentum = 0.5
epoch_num = 10
seed = 1
train_batch_size = 64
test_batch_size = 1000
dataset = 'awgn' # awgn, motion-blur, reduced-contrast-and-awgn
data_path = './data/n-mnist/'
model_save_path = '../cnn_trained/'

dataset = 'mnist' # 

if dataset == 'mnist'




for net in net_list:
    print('=====================', net,'======================')
    # Training settings
    use_cuda = not False and torch.cuda.is_available()
    accuracy_list = []
    torch.manual_seed(seed)
    
    device = torch.device("cuda" if use_cuda else "cpu")
    
    kwargs = {}
    if use_cuda:
        torch.cuda.manual_seed(seed)
        kwargs = {'num_workers': 1,'pin_memory': True}
        
    # load the mnist dataset
    if dataset = 'mnist':
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./data/mnist', 
                           train=True, 
                           download=True,
                           transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])),
            batch_size=train_batch_size, shuffle=True, **kwargs)
    
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./data/mnist',
                           train=False,
                           download=True,
                           transform=transforms.Compose([transforms.ToTensor(),
                                                         transforms.Normalize((0.1307,), (0.3081,))])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    # load the n-mnist dataset
    elif dataset == 'awgn' or == 'motion-blur' or =='reduced-contrast-and-awgn':
        train_loader = torch.utils.data.DataLoader(
            NMNIST('./data/n-mnist',
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))]),
                   dataset=dataset),batch_size=train_batch_size, shuffle=True, **kwargs)
    
        test_loader = torch.utils.data.DataLoader(
            NMNIST('./data/n-mnist',
                   train=False,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))]),
                   dataset=dataset), batch_size=test_batch_size, shuffle=True, **kwargs)
    
    if net == 'dnn':
        model = DNNNet().to(device)
    elif net == 'scnn':
        model = SimpleCNNNet().to(device)
    elif net == 'cnn':
        model = CNNNet().to(device)

    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    for epoch in range(1, epoch_num + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        acc = test(model, device, test_loader)
        accuracy_list.append(acc)

    if True:
        if net == 'dnn':
            torch.save(model.state_dict(), model_save_path+'mnist_dnn_net_'+ dataset + '_' + str(neuros)+".pt")
        elif net == 'scnn':
            torch.save(model.state_dict(), model_save_path+"mnist_cnn_net_simple_"+ dataset + '_' + str(neuros)+".pt")
        elif net == 'cnn':
            torch.save(model.state_dict(), model_save_path+"mnist_cnn_net_"+ dataset+ '_' + str(neuros) +".pt")
    print(accuracy_list)
    accuracys.append(accuracy_list)

print(accuracys)

===================== dnn ======================
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305805
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.316052

Test set: Average loss: 0.3519, Accuracy: 8950/10000 (89.500%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.305354
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.533873

Test set: Average loss: 0.2987, Accuracy: 9109/10000 (91.090%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.338250
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.301545

Test set: Average loss: 0.2576, Accuracy: 9222/10000 (92.220%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.161575
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.185538

Test set: Average loss: 0.2287, Accuracy: 9313/10000 (93.130%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.270964
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.089987

Test set: Average loss: 0.2074, Accuracy: 9370/10000 (93.700%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.189466
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.140427

Test set: Average loss: 0.1924, Accuracy: 9427/10000 (9

/Users/shengdaolin_sh/dir_install/Aanconda3/anaconda3/envs/env_msc_project_py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.451275

Test set: Average loss: 0.2278, Accuracy: 9295/10000 (92.950%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.078366
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.059162

Test set: Average loss: 0.1523, Accuracy: 9513/10000 (95.130%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.088205
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.047769

Test set: Average loss: 0.1410, Accuracy: 9553/10000 (95.530%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.123802
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.101916

Test set: Average loss: 0.1094, Accuracy: 9656/10000 (96.560%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.131662
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.029959

Test set: Average loss: 0.0843, Accuracy: 9724/10000 (97.240%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.042148
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.111388

Test set: Average loss: 0.0781, Accuracy: 9759/10000 (97.590%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.035004
Train Epoch: 7 [32000/60000 (53%)]	Loss: